In [61]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 37.7 MB/s eta 0:00:00


In [6]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd /content/gdrive/MyDrive/Colab Notebooks/analysis_edu/etlPjt/codeset

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/analysis_edu/etlPjt/codeset


# 라이브러리 선언

In [62]:
import pandas as pd
from sqlalchemy import create_engine, inspect

In [12]:
import os
import glob

### 파라미터 정의

In [57]:
try:
    paramDf = pd.read_csv("./paramData.txt")
    # 기본값
    dataRootPath = "../dataset"
    # 경로 불러오기 (param_seg: folder_loc, param_value)
    targetColumn = ["param_value"]
    dataRootPath = paramDf.loc[ paramDf.param_seg=="folder_loc", targetColumn].values[0][0]
except Exception as e:
    print(e)

In [58]:
dataRootPath

'../dataset'

### 1. 데이터 불러오기 및 데이터 저장 (to MySQL)

In [87]:
### 1. os.listdir 활용
fileNameList = os.listdir(dataRootPath)
print( fileNameList )
eachFileName = fileNameList[0]
fileNamePath = os.path.join(dataRootPath, eachFileName)
eachDf = pd.read_csv(fileNamePath, encoding="ms949")
eachDbName = eachFileName.split(".")[0]

user = 'kopouser'
password = 'kopouser'
host = '13.124.134.168'
port = '3306'
database = 'kopo'

try:
    # SQLAlchemy 엔진 생성
    engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}')

    # 데이터프레임을 MySQL 데이터베이스의 테이블에 저장
    table_name = eachDbName
    eachDf.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
    print(f"data {eachFileName} saved in db as a {eachDbName}")
except Exception as e:
    print(e)

['직급.csv', '근무지.csv', '부서.csv', '사원.csv', 'customerdata_out.xlsx', 'Untitled.ipynb']
data 직급.csv saved in db as a 직급


In [90]:
### 테이블 이름 매핑!

In [92]:
mappingTable = pd.DataFrame( [[eachFileName, eachDbName]], columns = ["org_name","new_name"])